#### WIAA Seeding Projections
We have the 2023-2024 WI Girls Basketball tournament seeds. This will give us a chance to see if our predictions on the weights are correct. In this notebook we will:

- Use the data scraped from the WIAA website to produce winning percentages used in producing the seeds for the WIAA tournament
- Merge the winning percentage data from the 2022-2023 and 2023-2024 seasons with the seeds from those tournaments
- Get estimates of how percentages are weighted by running a random search of the parameter space
- Use a grid search to see if we can nail down the exact weights
- Use a logistic regression classifier to check to see if we can use that to project the seeds in the tournament

We will start by loading in the data and calculating the winning percentages.

In [1]:
# import statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from datetime import datetime
import string
import warnings
import re

warnings.simplefilter("ignore")

In [2]:
def get_results(year=2023):
    """
    get_results is a method that returns a pandas DataFrame with every teams':
    - Team (name)
    - Enrollment
    - Website from the WIAA website that we pulled every teams' results from
    - A list of the Teams each Team played during the season
    - A list of the results from each game played - these will align with the order of teams listed
    - A list of dates each Team played - again, this will align with the list of teams played in order

    year: an int, this should be the year that begins each season (e.g. 2023 is for 2023-24 season); only
          allowable values for now are 2021 through 2023

    returns a pandas DataFrame - results
    """
    # Check year condition - if outside of allowable range - return an empty DataFrame
    if int(year) < 2021 or int(year) > 2023:
        results = pd.DataFrame()
    # If year in range of 2021 to 2023
    else:
        results = pd.read_csv(f"/Users/edebroux/Documents/GHS Basketball/gibraltar_basketball/wiaaWebscrape/wiaa{year}.csv", header=None).rename(columns={
            0: "Team",
            1: "Enrollment",
            2: "Website",
            3: "Teams_Played",
            4: "Results",
            5: "Dates"
        }).sort_values(by="Team").reset_index(drop=True)
        # Fix some names for consistency
        results["Team"] = results["Team"].str.replace("McDonell Central Catholic", "McDonell Catholic")
        results["Team"] = results["Team"].str.replace("Independence/Gil.", "Independence/Gilmanton")
        results["Team"] = results["Team"].str.replace("Stevens Point Area Senior High", "Stevens Point")
        results["Team"][results["Team"].str.contains("Racine St.")] = "Racine St. Catherines"
        # Same thing for teams played
        results["Teams_Played"] = results["Teams_Played"].str.replace("McDonell Central Catholic", "McDonell Catholic")
        results["Teams_Played"] = results["Teams_Played"].str.replace("Independence/Gil.", "Independence/Gilmanton")
        results["Teams_Played"] = results["Teams_Played"].str.replace("Stevens Point Area Senior High", "Stevens Point")
        results["Teams_Played"] = results["Teams_Played"].str.replace("Racine St. Catherine's", "Racine St. Catherines")
        results = results.drop_duplicates(keep="last").reset_index(drop=True)
    # Return results DataFrame
    return results

def get_tournament_seeds(year=None):
    """
    get_tournament_seeds is a method that returns the tournament seeds from the file of tournament seeds from the 
    2022, 2023, and 2024 versions of the WIAA girls state tournament. This file will have:
    - Team (name)
    - Division of the team for that season of the tournament
    - Sectional of the team for that season of the tournament
    - Seed of the team for that season of the tournment
    - Year of tournament for each team
    year: an int, this should be the year that ends each season (e.g. 2024 is for 2023-24 season); only
          allowable values for now are 2022 through 2024. Defaults to None

    returns a pandas DataFrame - tournament seeds
    """
    # Load in the tournament seeds for each year
    tourney = pd.read_excel("/Users/edebroux/Documents/GHS Basketball/gibraltar_basketball/wiaaWebscrape/Tournament Seeds 2022 and 2023 WI GBB.xlsx").iloc[1:, :]
    # If no year is entered
    if year is None:
        return tourney
    else:
        if int(year) == 2022:
            t22 = tourney[tourney.Year == 2022].reset_index(drop=True)
            return t22
        elif int(year) == 2023:
            t23 = tourney[tourney.Year == 2023].reset_index(drop=True)
            return t23
        elif int(year) == 2024:
            t24 = tourney[tourney.Year == 2024].reset_index(drop=True)
            return t24
        else:
            return tourney

def merge_tournament(year=2024):
    """
    merge_tournament is a method that tries to merge the results DataFrame and the tournament seeds DataFrame

    year: an int; the year of the tournament you are analyze

    returns merged DataFrame with the tournament seeds and results, and a list of teams with a tournament seed but missing records
    """
    # Get the results
    results = get_results(year=int(year)-1)
    # Clean backslash and hyphens
    # results["Team"] = results['Team'].str.replace('-', ' ', regex=True)
    # results["Team"] = results['Team'].str.replace('/', ' ', regex=True)
    # # Clean team names in results DataFrame
    # results["Team"] = results['Team'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
    # # Get the tournament seeds
    tourney = get_tournament_seeds(year=year)
    # # Clean backslash and hyphens
    # tourney["Team"] = tourney['Team'].str.replace('-', ' ', regex=True)
    # tourney["Team"] = tourney['Team'].str.replace('/', ' ', regex=True)
    # # Clean team names in results DataFrame
    # tourney["Team"] = tourney["Team"].str.replace('[{}]'.format(string.punctuation), '', regex=True)
    # Merge tournament results
    tr = results.merge(right=tourney, on="Team", how="outer")
    # Handle Racine St. Catherine's case - hoping removing punctuation helps
    # rsc = tr.index[tr["Team"].str.contains("Racine St.")].tolist()
    # # Overwrite the first row with Racine St. Catherine's tournament data if there is more than 1 row with Racine St. Catherine's data
    # if len(rsc) > 1:
    #     tr.iloc[int(rsc[0]), 6:] = seeds_results_23.iloc[int(rsc[1]), 6:]
    # else:
    #     pass
    # Check for any Teams with no game results, but have tournament seeds
    missing_results = tr[(tr["Enrollment"].isnull()==True)]["Team"].tolist()
    # Return the merged DataFrame and the list of teams with missing results
    return tr, missing_results

def clean_teams_played(team_list):
    """
    clean_teams_played takes a list of teams that were played by a team in a season and 
    cleans the list by splitting the team names and removing any unnecessary characters

    team_list: a str; this would be the list of teams a team played in a season

    returns a list of strings that are the teams played by a team in a given season
    """
    # Split the list of teams
    teams_played_all = str(team_list).split("', ")
    # Clean the strings by looping throgh the list
    for i in range(len(teams_played_all)):
        teams_played_all[i] = str(teams_played_all[i]).replace("[", "")
        teams_played_all[i] = str(teams_played_all[i]).replace("]", "")
        teams_played_all[i] = str(teams_played_all[i]).replace("'", "")
    # Return the cleaned list of teams played
    return teams_played_all

def clean_game_scores(game_list):
    """
    clean_game_scores is a method that cleans a list of scores for a team in a given year

    game_list: a str; this is a string that is written as a list that we use to get the
    list of scores for each game a team plays

    retrns a list of strings that contains the scores
    """
    # Split the list of scores
    game_scores = str(game_list).split(", ")
    # Loop throgh and collect the game scores
    for i in range(len(game_scores)):
        game_scores[i] = str(game_scores[i]).replace("[", "")
        game_scores[i] = str(game_scores[i]).replace("]", "")
        game_scores[i] = str(game_scores[i]).replace("'", "")
    # Return the cleaned list of scores from each game played
    return game_scores

def clean_dates(date_list, year):
    """
    clean_dates is a method that takes the list of strings pulled from the WIAA website
    and converts them to a list of datetime objects

    date_list: a str; this is a list casted to a string that would have all the dates
    that a game was played on
    year: an int; this would be the year of the tournament. Primarily used if a date 
    does not get cast to a datetime object

    returns a list of datetime objects
    """
    # Split the list of dates
    date_of_games = str(date_list).split(", ")
    # Clean dates and convert to datetime object
    for i in range(len(date_of_games)):
        date_of_games[i] = str(date_of_games[i]).replace("[", "")
        date_of_games[i] = str(date_of_games[i]).replace("]", "")
        date_of_games[i] = str(date_of_games[i]).replace("'", "")
        # Try casting to a datetime object
        try:
            date_of_games[i] = datetime.strptime(str(date_of_games[i]), '%m/%d/%Y')
        except ValueError:
            date_of_games[i] = datetime.strptime(f"04/01/{str(year)}", '%m/%d/%Y')
    # Return the list of dates games were played
    return date_of_games

def calculate_records(df, year):
    """
    calculate_records is a method that goes throgh and calculates the record of each team
    during the regular season prior to the seeding date

    df: a pandas DataFrame; the DataFrame will contain all the data pulled for every team
    in the WIAA's website belonging to a recognized conference or WIAA GBB team
    year: an int; the year of the WIAA GBB tournament (e.g. year 2024 would be the tournament
    for the 2023-24 season)

    returns a DataFrame with a boolean of whether a team won, list of points scored by the team in every game,
    list of points scored against every team in every game, a boolean list of games played prior to
    the seeding of the tournament, a boolean list of tournament games
    """
    # Create seeding data DataFrame
    seeding_data = pd.DataFrame({
        "Team": [], # Team name
        "Game_Dates": [], # list of datetime objects with the date of each game
        "Teams_Played": [], # ordered list of strings that contain the names of each teams' opponents
        "Win": [], # list of boolean lists of whether a team won or not
        "Points_For": [], # List of points scored by the team in every game
        "Points_Against": [], # List of points scored against the team in every game
        "Seed_Game": [], # Boolean list that tells us if a game is included in seeding
        "Tourney_Game": [] # Boolean list that tells us if a game is a tournament game
    })
    # Set the data types for each column
    seeding_data["Team"] = seeding_data["Team"].astype(object)
    seeding_data["Game_Dates"] = seeding_data["Game_Dates"].astype(object)
    seeding_data["Teams_Played"] = seeding_data["Teams_Played"].astype(object)
    seeding_data["Win"] = seeding_data["Win"].astype(object)
    seeding_data["Points_For"] = seeding_data["Points_For"].astype(object)
    seeding_data["Points_Against"] = seeding_data["Points_Against"].astype(object)
    seeding_data["Seed_Game"] = seeding_data["Seed_Game"].astype(object)
    seeding_data["Tourney_Game"] = seeding_data["Tourney_Game"].astype(object)
    # Loop throught the DataFrame and start collecting data for each record
    for i in range(len(df)):
        # Set the lists for the seeding DataFrame row i
        win = []
        pts_for = []
        pts_agn = []
        sg = []
        tg = []
        # Collect this data... start by setting the initial team...
        team = str(df.at[ i, "Team"])
        # print off the team's record being calculated - a "verbose" method
        print("Calculating record for: " + str(team))
        # ... and cleaning the list of teams played...
        teams_played = clean_teams_played(df.at[ i, "Teams_Played"])
        # ... and the scores from each game...
        game_scores = clean_game_scores(df.at[ i, "Results"])
        # ... and the dates of each game...
        date_of_games = clean_dates(df.at[ i, "Dates"], year=year)
        # Add entries to row i of seeding data
        # Start by getting the first day of the tournament indicator and
        # seeding game indicator cutoffs
        if year > 2024 or year < 2022:
            print("Not a tournament year for which we have data.")
        elif year == 2022:
            tournament_game = datetime(2022, 2, 22, 0, 0) #Tourney started 2/22/22
            seeding_game = datetime(2022, 2, 13, 0, 0) #Games after 2/13/22 not considered in seeding (mostly :/)
        elif year == 2023:
            tournament_game = datetime(2023, 2, 21, 0, 0) #Tourney started 2/21/23
            seeding_game = datetime(2023, 2, 12, 0, 0) #Games after 2/12/23 not considered in seeding
        else:
            tournament_game = datetime(2024, 2, 20, 0, 0) #Tourney started 2/20/24
            seeding_game = datetime(2024, 2, 11, 0, 0) #Games after 2/11/24 not considered in seeding
        # Now, let's start to load in our data
        for j in range(len(teams_played)):
            # Tournament game indicator
            if tournament_game <= date_of_games[j]:
                tg.append(True)
            else:
                tg.append(False)
            # Seeding game indicator
            if seeding_game <= date_of_games[j]:
                sg.append(False)
            else:
                sg.append(True)
            # # Add seeding game indicators if mismatching lengths
            # if len(sg) < len(tg):
            #     sg.append(False)
            # Append the score from each game to the score for and against array
            try:
                if (j != (len(df.at[i, "Dates"])-1)) and (int(year) == 2022) and (tg[j] == True):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2022) and (tg[j] == True) and (df.at[i, "Team"] not in ["Kettle Moraine", "Notre Dame", "Waupun", "Mineral Point", "Randolph"]):
                    pts_for.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2022) and (df.at[i, "Team"] in ["Kettle Moraine", "Notre Dame", "Waupun", "Mineral Point", "Randolph"]):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j != (len(df.at[i, "Dates"])-1)) and (int(year) == 2023) and (tg[j] == True):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2023) and (tg[j] == True) and (df.at[i, "Team"] not in ["Kettle Moraine", "Notre Dame", "Kewauskum", "Laconia", "McDonell Catholic"]):
                    pts_for.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2023) and (tg[j] == True) and (df.at[i, "Team"] in ["Kettle Moraine", "Notre Dame", "Kewauskum", "Laconia", "McDonell Catholic"]):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j != (len(df.at[i, "Dates"])-1)) and (tg[j] == True) and (int(year) == 2024):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2024) and (tg[j] == True) and (df.at[i, "Team"] not in ["Arrowhead", "Pewaukee", "Edgewood", "Laconia", "Albany Monticello"]):
                    pts_for.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                elif (j == (len(df.at[i, "Dates"])-1)) and (int(year) == 2024) and (tg[j] == True) and (df.at[i, "Team"] in ["Arrowhead", "Pewaukee", "Edgewood", "Laconia", "Albany Monticello"]):
                    pts_for.append(max(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored from game j for team in the list
                    pts_agn.append(min(int(game_scores[j].split("-")[0]), int(game_scores[j].split("-")[1]))) # Append points scored against a team from game j
                else:
                    pts_for.append(int(game_scores[j].split("-")[0])) # Append points scored from game j for team in the list
                    pts_agn.append(int(game_scores[j].split("-")[1])) # Append points scored against a team from game j
            except ValueError:
                pts_for = []
                pts_agn = []
            # Win-loss indicator based on appending the points for and against comparison
            try:
                if int(pts_for[j]) > int(pts_agn[j]):
                    win.append(True)
                else:
                    win.append(False)
            except ValueError:
                win = []
            except IndexError:
                win = []

        # Check to see if seeding game boolean is shorter than the date of games
        if len(sg)+1 == len(date_of_games):
            sg.append(False)
        # Okay, let's start compiling our seeding data:
        seeding_data.at[i, "Team"] = str(team)
        # Set these columns as objects
        seeding_data.at[i, "Win"] = win
        seeding_data.at[i, "Teams_Played"] = teams_played
        seeding_data.at[i, "Game_Dates"] = date_of_games
        seeding_data.at[i, "Points_For"] = pts_for
        seeding_data.at[i, "Points_Against"] = pts_agn
        seeding_data.at[i, "Seed_Game"] = sg
        seeding_data.at[i, "Tourney_Game"] = tg
    # Once we have looped through and calculated all the data, return the DataFrame
    return seeding_data

def get_winning_percentages(df, year, stgco=True):
    """
    get_winning_percentage is a method that calculates a teams' winning percentage
    based on the data we pull from the WIAA website

    df: a pandas DataFrame; this should be the pandas DataFrame generated from
    the calculate_records method 
    stgco: a boolean; stands for seeding for tournament games considered only. If True,
    a record is calculated using only games considered for seeding
    year: an int; relevant if the we are calculating records from games that are not 
    tournament games.

    returns a DataFrame that has every teams' winning percentage
    """
    # Create a winning percentage column
    df["wp%"] = [None] * len(df)
    # Create win and loss columns
    df["Wins"] = [None] * len(df)
    df["Losses"] = [None] * len(df)
    # Check if we are only calculating the record based on games only used for seeding
    if stgco:
        # Loop through and calculate the records for each team
        for i in range(len(df)):
            # Loop through and calculate the records
            # Set counters
            wins = 0
            games_played = 0
            for j in range(len(df.at[i, "Win"])):
                if list(df.at[i, "Seed_Game"])[j] == True:
                    games_played += 1
                    if list(df.at[i, "Win"])[j] == True:
                        wins += 1
                    else:
                        continue
                else:
                    continue
            # Calculate the win percentage
            try:
                df.at[i, "w%"] = round(float(wins)/float(games_played), 6)
                df.at[i, "Wins"] = wins
                df.at[i, "Losses"] = games_played - wins
            except ZeroDivisionError:
                df.at[i, "w%"] = 0.
                df.at[i, "Wins"] = wins
                df.at[i, "Losses"] = games_played - wins
    # Sum the # of wins
    else:
        # Loop through and calculate every teams' winning percentage if we are considering all games played
        # This is trickier due to tournament game scores have different formats
        for i in range(len(df)):
            # Consider case where a game a team plays is not a tournament game
            # Set counters
            wins = 0
            games_played = 0
            # Loop through and calculate the number of wins
            for j in range(len(df.at[i, "Win"])):
                # Add games_played
                games_played += 1
                # Case where a game is not a tourney game
                if list(df.at[i, "Tourney_Game"])[j] == False:
                    # If win is True
                    if list(df.at[i, "Win"])[j] == True:
                        wins += 1
                    else:
                        continue
                # If we are handling the case where there is a tournament game
                else:
                    # Check if there is a tournament game
                    if (j != (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2022):
                        wins += 1
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2022) and (df.at[i, "Team"] not in ["Kettle Moraine", "Notre Dame", "Waupun", "Mineral Point", "Randolph"]):
                        continue
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2022) and (df.at[i, "Team"] in ["Kettle Moraine", "Notre Dame", "Waupun", "Mineral Point", "Randolph"]):
                        wins += 1
                    elif (j != (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2023):
                        wins += 1
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2023) and (df.at[i, "Team"] not in ["Kettle Moraine", "Notre Dame", "Kewauskum", "Laconia", "McDonell Catholic"]):
                        continue
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2023) and (df.at[i, "Team"] in ["Kettle Moraine", "Notre Dame", "Kewauskum", "Laconia", "McDonell Catholic"]):
                        wins += 1
                    elif (j != (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2024):
                        wins += 1
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2024) and (df.at[i, "Team"] not in ["Arrowhead", "Pewaukee", "Edgewood", "Laconia", "Albany Monticello"]):
                        continue
                    elif (j == (len(df.at[i, "Tourney_Game"])-1)) and (int(year) == 2024) and (df.at[i, "Team"] in ["Arrowhead", "Pewaukee", "Edgewood", "Laconia", "Albany Monticello"]):
                        wins += 1
                    else:
                        print("Either the year is out of range, or something went wrong on the processing side and I missed a condition.")
            # Calculate the win percentage
            try:
                df.at[i, "w%"] = round(float(wins)/float(games_played), 6)
                df.at[i, "Wins"] = wins
                df.at[i, "Losses"] = games_played - wins
            except ZeroDivisionError:
                df.at[i, "w%"] = 0.
                df.at[i, "Wins"] = 0. 
                df.at[i, "Losses"] = 0.
    # Return the DataFrame with the calculated win percentage, number of wins, and number of losses
    return df 

def get_opp_winning_percentages(df, year, stgco=True):
    """
    get_opp_winning_percentages is a method that calculates the number of wins
    and losses each teams' opponents and defeated opponents accumulated over a season.
    
    df: a pandas DataFrame; this is a pandas DataFrame that has passed through
    the get_winning_percentages method prior to running this method. Every teams'
    winning percentage that they play needs to be in the DataFrame.
    stgco: a boolean; stands for seeding for tournament games considered only. If True,
    a record is calculated using only games considered for seeding
    year: an int; this would be the year we are calculating the winning percentage
    metrics for

    returns a pandas DataFrame with every teams' opponents' winning percentages and
    defeated opponents' winning percentages calculated
    """
    # Set counts for opponents wins and losses
    df["Opponent_Wins"] = 0.
    df["Opponent_Losses"] = 0. 
    df["Defeated_Opponent_Wins"] = 0.
    df["Defeated_Opponent_Losses"] = 0. 
    # Loop through and grab the teams played
    for i in range(len(df)):
        # Verbose method to track team record being calculated
        # print("Calculating opponent record of: " + str(df.at[i, "Team"]))
        # Clean the team names
        teams_played_all = clean_teams_played(df.at[i, "Teams_Played"])
        # Get a unique list of teams - no double counting W-L
        teams_played_set = set()
        # Get date of games
        #date_of_games = clean_dates(df.at[i, "Game_Dates"], year=year)
        # Compare length of these lists
        # if len(list(df.at[i, "Seed_Game"])) < len(list(df.at[i, "Game_Dates"])):
        #     df.at[i, "Seed_Game"] = list(df.at[i, "Seed_Game"]).insert(-1, False)
        # elif len(list(df.at[i, "Seed_Game"])) < len(list(df.at[i, "Game_Dates"])):
        #     df.at[i, "Game_Dates"] = list(df.at[i, "Game_Dates"]).append(datetime.strptime(f"04/01/{str(year)}", '%m/%d/%Y'))
        # print("Length of Seed Game Boolean list: " + str(len(df.at[i, "Seed_Game"])))
        # print("Length of Date of Games List: " + str(len(df.at[i, "Game_Dates"])))
        # Get the team name
        for j in range(len(df.at[i, "Game_Dates"])):
            # If considering only games used to seed the tournament
            if stgco and list(df.at[i, "Seed_Game"])[j] == True:
                try:
                    # Check if the opponent is not in the set of unique teams
                    if teams_played_all[j] not in teams_played_set:
                        df.at[i, "Opponent_Wins"] += float(df[df["Team"] == str(teams_played_all[j])]["Wins"].values[0])
                        df.at[i, "Opponent_Losses"] += float(df[df["Team"] == str(teams_played_all[j])]["Losses"].values[0])
                        # Check if the team won or lost to calculate defeated opponent W-L record
                        if list(df.at[i, "Win"])[j] == True:
                            df.at[i, "Defeated_Opponent_Wins"] += float(df[df["Team"] == str(teams_played_all[j])]["Wins"].values[0])
                            df.at[i, "Defeated_Opponent_Losses"] += float(df[df["Team"] == str(teams_played_all[j])]["Losses"].values[0])
                        else:
                            continue
                        # Add team to list of teams played
                        teams_played_set.add(teams_played_all[j])
                    else:
                        break
                # Out of state consideration
                except IndexError:
                    df.at[i, "Opponent_Wins"] += 10.
                    df.at[i, "Opponent_Losses"] += 10.
                    # Handle case where the team won
                    if list(df.at[i, "Win"])[j] == True:
                        df.at[i, "Defeated_Opponent_Wins"] += 10.
                        df.at[i, "Defeated_Opponent_Losses"] += 10.
                    else:
                        break
            # Case where we are only considering seeding games, but the game did
            # not count towards seeding
            elif stgco and list(df.at[i, "Seed_Game"])[j] == False:
                continue
            # If calculating opponent Win % based on all games played
            else:
                try:
                    if teams_played_all not in teams_played_set:
                        # Check if the opponent is not in the set of unique teams
                        df.at[i, "Opponent_Wins"] += float(df[df["Team"] == str(teams_played_all[j])]["Wins"].values[0])
                        df.at[i, "Opponent_Losses"] += float(df[df["Team"] == str(teams_played_all[j])]["Losses"].values[0])
                        # Calculation when if the team won against Team j
                        if list(df.at[i, "Win"])[j] == True:
                                df.at[i, "Defeated_Opponent_Wins"] += float(df[df["Team"] == str(teams_played_all[j])]["Wins"].values[0])
                                df.at[i, "Defeated_Opponent_Losses"] += float(df[df["Team"] == str(teams_played_all[j])]["Losses"].values[0])
                        else:
                            continue
                        # Add team to a list of teams played
                        teams_played_set.add(teams_played_all[j])
                    else:
                        continue
                # Out of state case
                except IndexError:
                    df.at[i, "Opponent_Wins"] += 10.
                    df.at[i, "Opponent_Losses"] += 10.
                    #  Handle case where the team won
                    if list(df.at[i, "Win"])[j] == True:
                        df.at[i, "Defeated_Opponent_Wins"] += 10.
                        df.at[i, "Defeated_Opponent_Losses"] += 10.
                    else:
                        continue
    # Calculate the Opponent Win % and Defeated Opponent Win %
    try:
        df["ow%"] = round((df["Opponent_Wins"]) / (df["Opponent_Wins"] + df["Opponent_Losses"]), 6)
        df["dow%"] = round((df["Defeated_Opponent_Wins"]) / (df["Defeated_Opponent_Wins"] + df["Defeated_Opponent_Losses"]), 6)
    except ZeroDivisionError:
        df["ow%"] = 0. 
        df["dow%"] = 0.
    # Return the DataFrame with the calculated opponent win %
    return df

def get_opp_opp_winning_percentages(df, year, stgco=True, noos=False):
    """
    get_opp_opp_winning_percentages is a method that calculates the number of wins
    and losses from each teams' list of opponents' opponents and the defeated opponents'
    opponents' winning percentages
    
    df: a pandas DataFrame; this is a pandas DataFrame that has passed through
    the get_opp_winning_percentages method prior to running this method. Every teams'
    winning percentage that they play needs to be in the DataFrame.
    year: an int; this would be the year we are calculating the winning percentage
    metrics for
    stgco: a boolean; stands for seeding for tournament games considered only. If True,
    a record is calculated using only games considered for seeding
    noos; a boolean; stands for no out-of-state. Means that any game played against an
    out-of-state team is not added to the opponents' opponents' win and loss total.
    Defaults to False since I am pretty sure the WIAA does not use these opponents' 
    records in their tabulations. If true, we imagine they play 20 games and their opponents
    play 20 games, so their opponents win half their games

    returns a pandas DataFrame with every teams' opponents' opponents winning percentages
    and defeated opponents' opponents winning percentages calculated
    """
    # Set counts for opponents wins and losses
    df["Opponent_Opponent_Wins"] = 0.
    df["Opponent_Opponent_Losses"] = 0. 
    df["Defeated_Opponent_Opponent_Wins"] = 0. 
    df["Defeated_Opponent_Opponent_Losses"] = 0.
    # Loop through and calculate Opponents' Opponents' Wins and Losses
    for i in range(len(df)):
        # Clean the team names
        teams_played_all = clean_teams_played(df.at[i, "Teams_Played"])
        # Get a unique list of teams - no double counting W-L
        teams_played_set = set()
        # Get the opponent name and calculate the wins and losses
        for j in range(len(df.at[i, "Game_Dates"])):
            # If considering only games used to seed the tournament
            if stgco:
                try:
                    # Check if the game is a seed game
                    if list(df.at[i, "Seed_Game"])[j] == True:
                        # Check if the opponent is not in the set of unique teams
                        if teams_played_all[j] not in teams_played_set:
                            df.at[i, "Opponent_Opponent_Wins"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Wins"].values[0]
                            df.at[i, "Opponent_Opponent_Losses"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Losses"].values[0]
                            # Calculation when if the team won against Team j
                            if list(df.at[i, "Win"])[j] == True:
                                    df.at[i, "Defeated_Opponent_Opponent_Wins"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Wins"].values[0]
                                    df.at[i, "Defeated_Opponent_Opponent_Losses"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Losses"].values[0]
                            else:
                                continue
                            # Add team to a list of teams played
                            teams_played_set.add(teams_played_all[j])
                        else:
                            continue
                except IndexError:
                    if noos:
                        df.at[i, "Opponent_Opponent_Wins"] += 100.
                        df.at[i, "Opponent_Opponent_Losses"] += 100.
                    else:
                        continue
            # Case if seed game is not True
            else:
                try:
                    if teams_played_all not in teams_played_set:
                        # Check if the opponent is not in the set of unique teams
                        df.at[i, "Opponent_Opponent_Wins"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Wins"].values[0]
                        df.at[i, "Opponent_Opponent_Losses"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Losses"].values[0]
                        # Calculation when if the team won against Team j
                        if list(df.at[i, "Win"])[j] == True:
                                df.at[i, "Defeated_Opponent_Opponent_Wins"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Wins"].values[0]
                                df.at[i, "Defeated_Opponent_Opponent_Losses"] += df[df["Team"] == str(teams_played_all[j])]["Opponent_Losses"].values[0]
                        else:
                            continue
                        teams_played_set.add(teams_played_all[j])
                    else:
                        continue
                except IndexError:
                    # If considering out-of-state games in this calculation
                    if noos:
                        df.at[i, "Opponent_Wins"] += 100.
                        df.at[i, "Opponent_Losses"] += 100.
                    else:
                        continue
    # Calculate the opponents' opponents' winning percentage and defeated opponents' opponents' winning percentage
    try:
        df["oow%"] = round((df["Opponent_Opponent_Wins"]) / (df["Opponent_Opponent_Wins"] + df["Opponent_Opponent_Losses"]), 6)
        df["doow%"] = round((df["Defeated_Opponent_Opponent_Wins"]) / (df["Defeated_Opponent_Opponent_Wins"] + df["Defeated_Opponent_Opponent_Losses"]), 6)
    except ZeroDivisionError:
        df["oow%"] = 0.
        df["doow%"] = 0. 
    # Return the DataFrame with the calculated opponent opponent win % and defeated opponent opponent win %
    return df

def return_winning_percentage_dfs(df, year):
    """
    return_winnning_percentage_dfs is a method that returns the:
    - Team Name
    - Seed
    - Division
    - Sectional
    - Seed
    - Year
    - winning percentages

    df: a pandas DataFrame; this should be a pandas DataFrame with all the calculated winning
    percentage data
    year: an int; should be between 2022 and 2024 and is the year of the tournament data was
    created with

    returns a subset with the winning percentages
    """
    # Grab tourney data
    tournament = get_tournament_seeds(year=year)
    # Merge tournament data
    percentages = df.merge(right=tournament, on="Team", how="outer")
    # Returns a DataFrame with the columns listed
    return percentages[["Team", "Division", "Sectional", "Seed", "Year", "w%", "ow%", "oow%", "dow%", "doow%"]]

def return_records_df(df, year):
    """
    return_winnning_percentage_dfs is a method that returns the:
    - Team Name
    - Seed
    - Division
    - Sectional
    - Seed
    - Year
    - Records for each metric

    df: a pandas DataFrame; this should be a pandas DataFrame with all the calculated winning
    percentage data
    year: an int; should be between 2022 and 2024 and is the year of the tournament data was
    created with

    returns a subset with the records:
    - Every teams' Win-Loss record
    - Every teams' Opponents' Win-Loss record
    - Every teams' Opponents' Opponents' Win-Loss record
    - Every teams' Defeated Opponents' Win-Loss record
    - Every teams' Defeated Opponents' Opponents' Win-Loss record
    """
    # Grab tourney data
    tournament = get_tournament_seeds(year=year)
    # Merge the tournament data and records data
    records = df.merge(right=tournament, on="Team", how="outer")
    # Returns a DataFrame with the columns listed
    return records[[
        "Team", 
        "Division", 
        "Sectional", 
        "Seed", 
        "Year",
        "Wins",
        "Losses",
        "Opponent_Wins",
        "Opponent_Losses",
        "Opponent_Opponent_Wins",
        "Opponent_Opponent_Losses",
        "Defeated_Opponent_Wins",
        "Defeated_Opponent_Losses",
        "Defeated_Opponent_Opponent_Wins",
        "Defeated_Opponent_Opponent_Losses"
    ]]

def return_elo_input_data(df, year):
    """
    return_elo_input_data is a method that outputs a subset of the calculated
    data used to calculate an adjusted ELO Rating system

    df: a pandas DataFrame; this should be the pandas DataFrame that contains
        all of our seeding data
    year: an int; should be between 2022 and 2024 and is the year of the tournament data was
    created with

    returns a DataFrame with:
    - Team
    - Division
    - Date of Games
    - Sectional
    - Seed
    - Year
    - Win Boolean
    - Points Scored
    - Points Against
    - Opponents
    """
    # Grab tourney data
    tournament = get_tournament_seeds(year=year)
    # Merge the tournament data and ELO-based data
    elo_input = df.merge(right=tournament, on="Team", how="outer")
    # Return the data used to calculate the ELO ratings
    return elo_input[[
        "Team", 
        "Division", 
        "Sectional", 
        "Seed", 
        "Year", 
        "Game_Dates",
        "Win", 
        "Points_For", 
        "Points_Against"
     ]]

Alright with all of our methods written out, let's see if our code works.

In [3]:
# Merge tournament seeds and results for 2023 tourney
tr_2223, missing_23 = merge_tournament(year=2023)
# Merge results and seeds for 2024 tourney
tr_2324, missing_24 = merge_tournament(year=2024)
# View DataFrame for 2023-24 season
tr_2324.head()

,Team,Enrollment,Website,Teams_Played,Results,Dates,Division,Sectional,Seed,Year
0,Abbotsford,244,https://schools.wiaawi.org/Directory/Schedule/...,"['Athens', 'Colby', 'Prentice', 'Greenwood', '...","['42-64', '69-29', '36-43', '47-48', '28-35', ...","['11/16/2023', '11/20/2023', '11/21/2023', '11...",4.0,1B,5.0,2024.0
1,Abundant Life Christian/St. Ambrose,211,https://schools.wiaawi.org/Directory/Schedule/...,"['Juda', 'Central WI Christian', 'Parkview', '...","['63-25', '45-28', '52-26', '39-73', '61-4', '...","['11/14/2023', '11/16/2023', '11/20/2023', '11...",4.0,4B,5.0,2024.0
2,Adams-Friendship,432,https://schools.wiaawi.org/Directory/Schedule/...,"['Cambridge', 'Poynette', 'Marshall', 'Amherst...","['34-65', '50-53', '36-45', '30-38', '40-54', ...","['11/14/2023', '11/16/2023', '11/20/2023', '11...",3.0,1B,9.0,2024.0
3,Albany/Monticello,178,https://schools.wiaawi.org/Directory/Schedule/...,"['Benton', 'Turner', 'Belleville', 'Williams B...","['73-26', '47-29', '55-37', '72-18', '69-56', ...","['11/14/2023', '11/16/2023', '11/18/2023', '11...",5.0,4B,1.0,2024.0
4,Algoma,219,https://schools.wiaawi.org/Directory/Schedule/...,"['Bonduel', 'Freedom', 'Mishicot', 'Green Bay ...","['22-45', '27-48', '40-48', '64-36', '32-45', ...","['11/14/2023', '11/16/2023', '11/27/2023', '11...",4.0,2B,6.0,2024.0


In [4]:
# View teams with missing records
missing_24

[]

In [5]:
missing_23

[]

In [6]:
# Calculate the records
seed24 = calculate_records(tr_2324, year=2024)

Calculating record for: Abbotsford
Calculating record for: Abundant Life Christian/St. Ambrose
Calculating record for: Adams-Friendship
Calculating record for: Albany/Monticello
Calculating record for: Algoma
Calculating record for: Alma Center Lincoln
Calculating record for: Alma/Pepin
Calculating record for: Almond-Bancroft
Calculating record for: Altoona
Calculating record for: Amery
Calculating record for: Amherst
Calculating record for: Antigo
Calculating record for: Appleton East
Calculating record for: Appleton North
Calculating record for: Appleton West
Calculating record for: Aquinas
Calculating record for: Arcadia
Calculating record for: Argyle/Pecatonica
Calculating record for: Arrowhead
Calculating record for: Ashland
Calculating record for: Ashwaubenon
Calculating record for: Assumption
Calculating record for: Athens
Calculating record for: Auburndale
Calculating record for: Augusta
Calculating record for: Badger
Calculating record for: Baldwin-Woodville
Calculating record

In [7]:
# Calculate the winning % with seeding games considered only
wp24_sgo = get_winning_percentages(df=seed24, year=2024, stgco=True)
wp24_sgo.head()

,Team,Game_Dates,Teams_Played,Win,Points_For,Points_Against,Seed_Game,Tourney_Game,wp%,Wins,Losses,w%
0,Abbotsford,"[2023-11-16 00:00:00, 2023-11-20 00:00:00, 202...","[Athens, Colby, Prentice, Greenwood, Auburndal...","[False, True, False, False, False, True, True,...","[42, 69, 36, 47, 28, 52, 45, 46, 72, 54, 37, 4...","[64, 29, 43, 48, 35, 39, 30, 40, 15, 21, 50, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,10,11,0.476190
1,Abundant Life Christian/St. Ambrose,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Juda, Central WI Christian, Parkview, Argyle/...","[True, True, True, False, True, True, False, T...","[63, 45, 52, 39, 61, 57, 34, 69, 54, 64, 69, 5...","[25, 28, 26, 73, 4, 29, 55, 33, 47, 10, 12, 12...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,16,5,0.761905
2,Adams-Friendship,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Cambridge, Poynette, Marshall, Amherst, Tomah...","[False, False, False, False, False, False, Tru...","[34, 50, 36, 30, 40, 44, 63, 62, 49, 39, 44, 6...","[65, 53, 45, 38, 54, 49, 32, 71, 75, 55, 62, 3...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,4,19,0.173913
3,Albany/Monticello,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Benton, Turner, Belleville, Williams Bay, Bel...","[True, True, True, True, True, True, True, Tru...","[73, 47, 55, 72, 69, 68, 90, 81, 65, 69, 78, 8...","[26, 29, 37, 18, 56, 38, 20, 15, 37, 41, 32, 2...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,20,3,0.869565
4,Algoma,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Bonduel, Freedom, Mishicot, Green Bay N.E.W. ...","[False, False, False, True, False, True, False...","[22, 27, 40, 64, 32, 51, 30, 56, 55, 43, 48, 4...","[45, 48, 48, 36, 45, 28, 37, 41, 34, 30, 34, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,12,10,0.545455


In [8]:
# Get opponent winning percentages
wp_owp_24_sgo = get_opp_winning_percentages(df=wp24_sgo, year=2024, stgco=True)
wp_owp_24_sgo.head()

,Team,Game_Dates,Teams_Played,Win,Points_For,Points_Against,Seed_Game,Tourney_Game,wp%,Wins,Losses,w%,Opponent_Wins,Opponent_Losses,Defeated_Opponent_Wins,Defeated_Opponent_Losses,ow%,dow%
0,Abbotsford,"[2023-11-16 00:00:00, 2023-11-20 00:00:00, 202...","[Athens, Colby, Prentice, Greenwood, Auburndal...","[False, True, False, False, False, True, True,...","[42, 69, 36, 47, 28, 52, 45, 46, 72, 54, 37, 4...","[64, 29, 43, 48, 35, 39, 30, 40, 15, 21, 50, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,10,11,0.476190,189.0,207.0,71.0,127.0,0.477273,0.358586
1,Abundant Life Christian/St. Ambrose,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Juda, Central WI Christian, Parkview, Argyle/...","[True, True, True, False, True, True, False, T...","[63, 45, 52, 39, 61, 57, 34, 69, 54, 64, 69, 5...","[25, 28, 26, 73, 4, 29, 55, 33, 47, 10, 12, 12...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,16,5,0.761905,81.0,91.0,46.0,83.0,0.470930,0.356589
2,Adams-Friendship,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Cambridge, Poynette, Marshall, Amherst, Tomah...","[False, False, False, False, False, False, Tru...","[34, 50, 36, 30, 40, 44, 63, 62, 49, 39, 44, 6...","[65, 53, 45, 38, 54, 49, 32, 71, 75, 55, 62, 3...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,4,19,0.173913,292.0,215.0,23.0,64.0,0.575937,0.264368
3,Albany/Monticello,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Benton, Turner, Belleville, Williams Bay, Bel...","[True, True, True, True, True, True, True, Tru...","[73, 47, 55, 72, 69, 68, 90, 81, 65, 69, 78, 8...","[26, 29, 37, 18, 56, 38, 20, 15, 37, 41, 32, 2...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,20,3,0.869565,150.0,165.0,114.0,159.0,0.476190,0.417582
4,Algoma,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Bonduel, Freedom, Mishicot, Green Bay N.E.W. ...","[False, False, False, True, False, True, False...","[22, 27, 40, 64, 32, 51, 30, 56, 55, 43, 48, 4...","[45, 48, 48, 36, 45, 28, 37, 41, 34, 30, 34, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,12,10,0.545455,138.0,167.0,47.0,127.0,0.452459,0.270115


In [9]:
wp_owp_oowp_24 = get_opp_opp_winning_percentages(df=wp_owp_24_sgo, year=2024)
wp_owp_oowp_24.head()

,Team,Game_Dates,Teams_Played,Win,Points_For,Points_Against,Seed_Game,Tourney_Game,wp%,Wins,...,Defeated_Opponent_Wins,Defeated_Opponent_Losses,ow%,dow%,Opponent_Opponent_Wins,Opponent_Opponent_Losses,Defeated_Opponent_Opponent_Wins,Defeated_Opponent_Opponent_Losses,oow%,doow%
0,Abbotsford,"[2023-11-16 00:00:00, 2023-11-20 00:00:00, 202...","[Athens, Colby, Prentice, Greenwood, Auburndal...","[False, True, False, False, False, True, True,...","[42, 69, 36, 47, 28, 52, 45, 46, 72, 54, 37, 4...","[64, 29, 43, 48, 35, 39, 30, 40, 15, 21, 50, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,10,...,71.0,127.0,0.477273,0.358586,3264.0,3112.0,1601.0,1561.0,0.511920,0.506325
1,Abundant Life Christian/St. Ambrose,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Juda, Central WI Christian, Parkview, Argyle/...","[True, True, True, False, True, True, False, T...","[63, 45, 52, 39, 61, 57, 34, 69, 54, 64, 69, 5...","[25, 28, 26, 73, 4, 29, 55, 33, 47, 10, 12, 12...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,16,...,46.0,83.0,0.470930,0.356589,2533.0,2476.0,1754.0,1669.0,0.505690,0.512416
2,Adams-Friendship,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Cambridge, Poynette, Marshall, Amherst, Tomah...","[False, False, False, False, False, False, Tru...","[34, 50, 36, 30, 40, 44, 63, 62, 49, 39, 44, 6...","[65, 53, 45, 38, 54, 49, 32, 71, 75, 55, 62, 3...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,4,...,23.0,64.0,0.575937,0.264368,3871.0,4263.0,428.0,463.0,0.475904,0.480359
3,Albany/Monticello,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Benton, Turner, Belleville, Williams Bay, Bel...","[True, True, True, True, True, True, True, Tru...","[73, 47, 55, 72, 69, 68, 90, 81, 65, 69, 78, 8...","[26, 29, 37, 18, 56, 38, 20, 15, 37, 41, 32, 2...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,20,...,114.0,159.0,0.476190,0.417582,3433.0,3255.0,2930.0,2748.0,0.513307,0.516027
4,Algoma,"[2023-11-14 00:00:00, 2023-11-16 00:00:00, 202...","[Bonduel, Freedom, Mishicot, Green Bay N.E.W. ...","[False, False, False, True, False, True, False...","[22, 27, 40, 64, 32, 51, 30, 56, 55, 43, 48, 4...","[45, 48, 48, 36, 45, 28, 37, 41, 34, 30, 34, 4...","[True, True, True, True, True, True, True, Tru...","[False, False, False, False, False, False, Fal...",None,12,...,47.0,127.0,0.452459,0.270115,2803.0,2819.0,1504.0,1409.0,0.498577,0.516306


In [10]:
input_data = return_winning_percentage_dfs(df=wp_owp_oowp_24, year=2024)
input_data.head()

,Team,Division,Sectional,Seed,Year,w%,ow%,oow%,dow%,doow%
0,Abbotsford,4.0,1B,5.0,2024.0,0.476190,0.477273,0.511920,0.358586,0.506325
1,Abundant Life Christian/St. Ambrose,4.0,4B,5.0,2024.0,0.761905,0.470930,0.505690,0.356589,0.512416
2,Adams-Friendship,3.0,1B,9.0,2024.0,0.173913,0.575937,0.475904,0.264368,0.480359
3,Albany/Monticello,5.0,4B,1.0,2024.0,0.869565,0.476190,0.513307,0.417582,0.516027
4,Algoma,4.0,2B,6.0,2024.0,0.545455,0.452459,0.498577,0.270115,0.516306


In [11]:
input_data.to_excel("/Users/edebroux/Documents/GHS Basketball/gibraltar_basketball/wiaaWebscrape/wiaa_model_inputs_2024.xlsx", index=False)